In [1]:
import csv
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [59]:
FULLDS = pd.read_csv("MergedDSFULL.csv")

C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
FULLDS.head()

,text_ID,start,end,object,value,unit,min,max,text
0,46809,0,6,RR,130,NaN,NaN,NaN,RR 130/80 mmHg.
1,43588,2421,2433,ALP,54,U/L,NaN,NaN,UURINGUD JA PROTSEDUURID \nLÄBIVAATUS: Objekt...
2,43050,720,735,HCT,40,%,NaN,41.2,UURINGUD JA PROTSEDUURID \n|Uuring|24.04|25.0...
3,45389,165,171,RR,110,NaN,NaN,NaN,"Kergelt liigkaaluline. Nahk, limaskestad taval..."
4,42960,2314,2332,HCT,45,%,40.0,50.0,UURINGUD JA PROTSEDUURID \nObjektiivselt: Nor...


In [63]:
FR_DS = FULLDS[FULLDS["object"] == "fr"]

In [65]:
FR_DS = FR_DS[FR_DS['text'].notna()]

In [66]:
FR_Subset = FR_DS[FR_DS['text'].apply(len)<150]


In [66]:
len(DS["text_ID"].unique())

4072

In [71]:
DS.head()

,text_ID,start,end,object,value,unit,min,max,text
8,51616,98,103,fr,72,NaN,NaN,NaN,\nNahaleid: puhas\nHingamiselundite leid: vesi...
26,101498,15,23,fr,83,x',NaN,NaN,"Cor regul rütm fr 83 x', toonid ii, RR 150/100..."
182,189919,105,110,fr,59,NaN,NaN,NaN,\nNahaleid: puhas\nHingamiselundite leid: vesi...
389,346549,84,90,fr,116,NaN,NaN,NaN,Praegusel hospitaliseerimisel patsiendi eneset...
593,546434,40,45,fr,70,NaN,NaN,NaN,\nDgn:M17.1; Z96.6; M48.0; M51.1 ...


In [95]:
DS = FR_Subset.copy()
new_df = pd.DataFrame(DS["text"].str.split().tolist(), index=DS.text_ID).stack()
new_df = new_df.reset_index([0, 'text_ID'])
new_df.columns = ['text_ID', 'word']
new_df["tag"]= np.NaN
myList= list(new_df.text_ID.unique())
for i in range(len(myList)):
    Original = DS.loc[DS["text_ID"] == myList[i]]
    words = new_df.loc[new_df["text_ID"] == myList[i]]
    for OriginalIndex, OriginalRow in Original.iterrows():
        for index, row in words.iterrows():
                if(str(Original["object"][OriginalIndex]) in str(words["word"][index])):
                    new_df["tag"][index] = "object"
                elif(not float(Original["value"][OriginalIndex]).is_integer()):
                    if(Original["value"][OriginalIndex] in str(words["word"][index])):
                        new_df["tag"][index] = "object"
                elif(str(int(Original["value"][OriginalIndex])) in str(words["word"][index])):
                    new_df["tag"][index] = "object"


C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Kasutaja\Anaconda3\envs\myEnv\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [90]:
not float(18).is_integer()

False

In [100]:
new_df[new_df["tag"]=="object"][50:100]

,text_ID,word,tag
564,32186,97,object
575,41846,fr,object
576,41846,"59x`,",object
589,45568,fr,object
590,45568,76,object
611,48451,fr,object
612,48451,"68x`,",object
624,49389,fr,object
625,49389,"78,",object
642,49874,fr,object


In [69]:
new_df.to_csv("FR_Object_tagged_Subset_150SentenceLength.csv",index=False)